In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

1. ['1. 디지털카메라', '2. 스마트패드', '3. 스마트폰'] 폴더 안에 <br> 들어있던 데이터 들을 ['01.원천데이터','02.라벨링데이터'] 폴더에 넣고 위 폴더들은 삭제

2. JSON형태의 라벨링 데이터를 DataFrame으로 변환

3. 데이터 프레임에 있는 None값 -> {}

4. 딕셔너리 안에 있는 None값 -> 0(임의로 설정)

In [ ]:
import os
import json
import shutil
import pandas as pd
from PIL import Image

devices_name = ['1. 디지털카메라', '2. 스마트패드', '3. 스마트폰']
# 1. 라벨링 데이터 정리
def reorganize_labeling_data(base_path):
    for split in ['Validation', 'Training']:
        label_path = os.path.join(base_path, split, '02.라벨링데이터')
        if not os.path.exists(label_path):
            print(f"경로가 존재하지 않습니다: {label_path}")
            continue
        for device in devices_name:
            device_path = os.path.join(label_path, device)
            if not os.path.exists(device_path):
                print(f"경로가 존재하지 않습니다: {device_path}")
                continue
            for root, dirs, files in os.walk(device_path):
                for file in files:
                    if file.endswith('.json'):
                        src = os.path.join(root, file)
                        dst = os.path.join(label_path, file)
                        shutil.move(src, dst)

        # 빈 폴더 제거
        for device in devices_name:
            device_path = os.path.join(label_path, device)
            if os.path.exists(device_path):
                shutil.rmtree(device_path)

In [ ]:
# 2. 이미지 데이터 정리
def reorganize_image_data(base_path):
    for split in ['Validation', 'Training']:
        image_path = os.path.join(base_path, split, '01.원천데이터')
        if not os.path.exists(image_path):
            print(f"경로가 존재하지 않습니다: {image_path}")
            continue
        for device in devices_name:
            device_path = os.path.join(image_path, device)
            if not os.path.exists(device_path):
                print(f"경로가 존재하지 않습니다: {device_path}")
                continue
            for root, dirs, files in os.walk(device_path):
                for file in files:
                    if file.endswith('.jpg'):
                        src = os.path.join(root, file)
                        dst = os.path.join(image_path, file)
                        shutil.move(src, dst)

        # 빈 폴더 제거
        for device in devices_name:
            device_path = os.path.join(image_path, device)
            if os.path.exists(device_path):
                shutil.rmtree(device_path)

In [ ]:
# 3. 라벨링 데이터를 DataFrame으로 변환
def create_labeling_dataframe(base_path):
    data = []
    for split in ['Validation', 'Training']:
        label_path = os.path.join(base_path, split, '02.라벨링데이터')
        if not os.path.exists(label_path):
            print(f"경로가 존재하지 않습니다: {label_path}")
            continue
        for file in os.listdir(label_path):
            if file.endswith('.json'):
                with open(os.path.join(label_path, file), 'r') as f:
                    json_data = json.load(f)
                    row = {
                        'split': split,
                        'info': json.dumps(json_data.get('info', {})),
                        'images': json.dumps(json_data.get('images', {})),
                        'annotations': json.dumps(json_data.get('annotations', {})),
                        'equipment': json.dumps(json_data.get('equipment', {}))
                    }
                    data.append(row)

    df = pd.DataFrame(data)
    return df

In [ ]:
# 4. 이미지 자르기 및 저장
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def crop_and_save_images(base_path):

    for split in ['Validation', 'Training']:
        image_path = os.path.join(base_path, split, '01.원천데이터')
        label_path = os.path.join(base_path, split, '02.라벨링데이터')

        if not os.path.exists(image_path) or not os.path.exists(label_path):
            print(f"경로가 존재하지 않습니다: {image_path} 또는 {label_path}")
            continue

        for facepart in range(9):
            os.makedirs(os.path.join(base_path, f'{split}_cropped', str(facepart)), exist_ok=True)

        for file in os.listdir(image_path):
            if file.endswith('.jpg'):
                try:
                    img = Image.open(os.path.join(image_path, file))
                    img.verify()  # 이미지 파일 검증
                    img = Image.open(os.path.join(image_path, file))  # 다시 열기
                except Exception as e:
                    print(f"오류 발생: {file} - {str(e)}")
                    continue

                for json_file in os.listdir(label_path):
                    if json_file.startswith(file[:-4]) and json_file.endswith('.json'):
                        with open(os.path.join(label_path, json_file), 'r') as f:
                            json_data = json.load(f)

                        facepart = json_data['images']['facepart']
                        bbox = json_data['images']['bbox']

                        try:
                            if bbox is None:
                                # bbox가 None이면 전체 이미지 사용
                                bbox = [0, 0, img.width, img.height]
                            else:
                                # bbox 좌표가 문자열인 경우 정수로 변환
                                bbox = [int(float(coord)) for coord in bbox]

                            # bbox 좌표가 올바른지 확인
                            if bbox[2] <= bbox[0] or bbox[3] <= bbox[1]:
                                print(f"잘못된 bbox 좌표: {file} - {bbox}")
                                continue

                            cropped_img = img.crop(bbox)
                            cropped_img.save(os.path.join(base_path, f'{split}_cropped', str(facepart), f"{file[:-4]}_{facepart}.jpg"))
                        except Exception as e:
                            print(f"이미지 크롭 오류 발생: {file} - {str(e)}")
                            continue

### 이미지와 폴더 정리는 제가 이미 다해놔서 돌릴 필요 없습니다!
- df = create_labeling_dataframe(base_path) 이것만 수행하면 됩니다.

In [ ]:
# 실행 코드
base_path = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터'  # 데이터 기본 경로

# reorganize_labeling_data(base_path)
# reorganize_image_data(base_path)
df = create_labeling_dataframe(base_path)
# crop_and_save_images(base_path)

# 5. 데이터 프레임 전처리
# JSON 문자열을 딕셔너리로 변환
for col in ['info', 'images', 'annotations', 'equipment']:
    df[col] = df[col].apply(lambda x: x if isinstance(x, dict) else json.loads(x) if isinstance(x, str) else x)

# 데이터 프레임 None 값을 {}로 변경
# 딕셔너리 안에 None 값을 0으로 바꾸는 함수 정의
def replace_none_with_zero(d):
    if isinstance(d, dict):
        return {k: (0 if v is None else v) for k, v in d.items()}
    return d

# 데이터 프레임의 모든 셀에 대해 함수 적용
df = df.applymap(replace_none_with_zero)

# 'equipment' 컬럼에 null 값을 빈 딕셔너리로 변경
df['equipment'] = df['equipment'].apply(lambda x: {} if pd.isna(x) else x)
df.head(3)

print("데이터 정리 및 처리가 완료!")
print(df.head(3))

##### 데이터 프레임 값이 딕셔너리를 문자열로 만들어 값에 들어가 있어 문자열 -> 원상태

In [ ]:
# YOLO 모델 로드 함수
def load_yolo_model(model_path):
    return YOLO(model_path)

In [ ]:
# 이미지를 224x224 크기로 로드 (ResNet50의 기본 입력 크기), 픽셀 값을 0~1 사이로 정규화
#  prepare_skin_data 함수에서 각 facepart 이미지를 로드할 때 사용
# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(img_path):
    img = load_img(img_path, target_size=(224, 224))  # ResNet50의 기본 입력 크기
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # 정규화
    return img_array

In [ ]:
# 피부 진단 데이터 준비 함수
def prepare_skin_data(df, facepart, base_path):
    # DataFrame에서 'Training' 데이터와 특정 facepart에 해당하는 데이터만 선택
    skin_data = df[(df['split'] == 'Training') & (df['images'].apply(lambda x: json.loads(x)['facepart'] == facepart))]

    X = []  # 이미지 데이터를 저장할 리스트
    y = []  # 라벨 데이터를 저장할 리스트

    for _, row in skin_data.iterrows():
        info = json.loads(row['info'])
        annotations = json.loads(row['annotations'])
        equipment = json.loads(row['equipment'])

        # 미리 자른 이미지 파일 경로
        img_path = os.path.join(base_path, 'Training_cropped', str(facepart), info['filename'])
        # 이미지 로드 및 전처리
        img = load_and_preprocess_image(img_path)
        X.append(img)

        # 라벨 데이터 준비
        if annotations is not None and equipment is not None:
            features = list(annotations.values()) + list(equipment.values())
            y.append(features)
        else:
            y.append([info['skin_type'], info['sensitive']])

    return np.array(X), np.array(y)

In [ ]:
# CNN 모델 생성 함수
def create_cnn_model(input_shape, output_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(output_shape, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# ResNet50 모델 생성 함수
def create_resnet50_model(input_shape, output_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(output_shape, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=x)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# 피부 진단 모델 학습 함수
# 'Training'폴더 데이터만 사용 'Validation'폴더에 있는 데이터들은 서비스 구현 테스트할 때 최종테스트용
def train_skin_models(df, base_path, model_type='cnn'):
    skin_models = {}

    for facepart in range(9):
        X, y = prepare_skin_data(df, facepart, base_path)

        if len(X) == 0:
            continue

        # Training 데이터를 train과 validation으로 분할
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state= 0)

        input_shape = X_train.shape[1:]
        output_shape = y_train.shape[1]

        if model_type == 'cnn':
            model = create_cnn_model(input_shape, output_shape)
        elif model_type == 'resnet50':
            model = create_resnet50_model(input_shape, output_shape)
        else:
            raise ValueError("Invalid model type. Choose 'cnn' or 'resnet50'.")

        # 이미지 데이터 증강
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True
        )

        # 모델 학습
        model.fit(datagen.flow(X_train, y_train, batch_size=32),
                  epochs=50,
                  validation_data=(X_val, y_val))

        skin_models[facepart] = model

    return skin_models

In [ ]:
# 이미지에서 얼굴 부위 추출 함수
def extract_faceparts(image, yolo_model):
    results = yolo_model(image)
    faceparts = []
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            facepart = image[int(y1):int(y2), int(x1):int(x2)]
            faceparts.append(facepart)
    return faceparts

In [ ]:
# 피부 상태 예측 함수
def predict_skin_condition(image_path, yolo_model, skin_models):
    image = load_img(image_path)
    image = img_to_array(image)

    faceparts = extract_faceparts(image, yolo_model)

    predictions = {}
    for i, facepart in enumerate(faceparts):
        facepart = load_and_preprocess_image(facepart)
        facepart = np.expand_dims(facepart, axis=0)

        if i in skin_models:
            prediction = skin_models[i].predict(facepart)
            predictions[i] = prediction[0]

    return predictions

In [ ]:
# 메인 실행 코드
if __name__ == "__main__":
    base_path = '/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터'  # 데이터 기본 경로
    yolo_model_path = 'path/to/your/trained_yolo_model.pt' # 학습해서 나오는 best yolo모델 path 입력

    # YOLO 모델 로드 (이미 학습된 모델 사용)
    yolo_model = load_yolo_model(yolo_model_path)

    # 피부 진단 모델 학습 (CNN 사용)
    skin_models_cnn = train_skin_models(df, base_path, model_type='cnn')

    # 피부 진단 모델 학습 (ResNet50 사용)
    skin_models_resnet = train_skin_models(df, base_path, model_type='resnet50')

    # 모델 저장
    for facepart, model in skin_models_cnn.items():
        model.save(f'skin_model_cnn_{facepart}.h5')
    for facepart, model in skin_models_resnet.items():
        model.save(f'skin_model_resnet_{facepart}.h5')

    print("모든 모델 학습 완료")


### 최종 테스트

In [ ]:
# Validation 데이터셋의 이미지들에 대해 예측 실행
validation_image_path = os.path.join(base_path, 'Validation', '01.원천데이터')
for image_file in os.listdir(validation_image_path):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(validation_image_path, image_file)
        predictions = predict_skin_condition(image_path, yolo_model, skin_models_cnn)  # 또는 skin_models_resnet
        print(f"이미지 {image_file}의 예측 결과:", predictions)